In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import zipfile
import os

def unzip_folder(zip_file_path, extract_to):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Example usage
zip_file_path = '/content/drive/MyDrive/Machine_learning/Deep_learning/Computer vision/Image Classification/Pepper.zip'
extract_to = './'
unzip_folder(zip_file_path, extract_to)


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
# Define paths
train_dir = '/content/Pepper/train'
validation_dir = '/content/Pepper/valid'
test_dir = '/content/Pepper/valid'

# Image data generator for training, validation and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 1731 images belonging to 2 classes.
Found 494 images belonging to 2 classes.
Found 494 images belonging to 2 classes.


In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


87910968/87910968 [==============================] - 5s 0us/step


In [6]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=4
)

Epoch 1/4
54/54 [==============================] - 56s 789ms/step - loss: 0.3736 - accuracy: 0.8428 - val_loss: 0.1236 - val_accuracy: 0.9521
Epoch 2/4
54/54 [==============================] - 42s 772ms/step - loss: 0.1300 - accuracy: 0.9541 - val_loss: 0.0822 - val_accuracy: 0.9688
Epoch 3/4
54/54 [==============================] - 42s 773ms/step - loss: 0.1210 - accuracy: 0.9535 - val_loss: 0.0655 - val_accuracy: 0.9729
Epoch 4/4
54/54 [==============================] - 45s 828ms/step - loss: 0.0922 - accuracy: 0.9635 - val_loss: 0.0998 - val_accuracy: 0.9604


In [7]:
class_name={v:k for k,v in train_generator.class_indices.items()}
class_name

{0: 'Pepper__bell___Bacterial_spot', 1: 'Pepper__bell___healthy'}

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

15/15 [==============================] - 2s 135ms/step - loss: 0.1011 - accuracy: 0.9583
Test loss: 0.10108127444982529
Test accuracy: 0.9583333134651184


In [9]:
model.save("./pepper_disease_classification_inception_v3_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

# Function to load and preprocess the image
def load_and_preprocess_image(img_path, target_size=(299, 299)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

class_name={v:k for k,v in train_generator.class_indices.items()}

# Path to the image you want to test
img_path = '/content/Pepper/test/Pepper__bell___healthy/e6184b41-1a46-47f9-bcc7-d9f16859e50d___JR_HL 8208.JPG'

# Load and preprocess the image
img_array = load_and_preprocess_image(img_path)

# Load the model (assuming the model is saved as 'model.h5')
model = tf.keras.models.load_model("./pepper_disease_classification_inception_v3_model.h5")

# Make a prediction
predictions = model.predict(img_array)

# Since it's binary classification, interpret the result
predicted_class = 1 if predictions[0] > 0.5 else 0
print(f"Predicted class: {class_name[predicted_class]}")

# Alternatively, print the prediction probabilities
print("Prediction probabilities:", predictions[0])


1/1 [==============================] - 3s 3s/step
Predicted class: Pepper__bell___healthy
Prediction probabilities: [0.9998764]
